# 01. KRX 원시 주가 데이터 수집

## 📋 이 단계의 책임

- ✅ pykrx API를 통한 원시 데이터 수집
- ✅ **API가 제공하는 모든 컬럼 보존** (등락률, 거래대금, 시가총액 등)
- ✅ 컬럼명 표준화 (한글 → 영문)
- ✅ CSV 파일로 저장 (종목별)

## ❌ 이 단계의 비책임

- ❌ **우리가 계산하는** Feature (이동평균, RSI, MACD 등)
- ❌ Target 계산 (수익률 변환, 로그 변환 등)
- ❌ 데이터 필터링 또는 결측치 처리

## 💡 핵심 원칙

1. **무판단 원칙**: "API가 주는 것은 전부 받는다"
   - KRX가 계산한 등락률 ✅ (그들이 계산한 것)
   - 우리가 계산할 RSI ❌ (우리가 계산할 것)

2. **단일 책임 원칙**: "이 단계는 '수집'만 한다"
   - 모든 가공은 `02_build_features.py`에서 수행

## 🔧 설정

In [ ]:
from src.data_loader.collector import RawPriceCollector, get_ticker_universe

In [ ]:
# ==================== 설정 ====================
# 수정이 필요한 경우 여기만 변경하세요

REFERENCE_DATE = "20251217"  # 종목 리스트 기준일
START_DATE = "20250101"      # 데이터 조회 시작일
END_DATE = "20251217"        # 데이터 조회 종료일

OUTPUT_DIR = f"data/01_raw/krx_prices_{REFERENCE_DATE}"

# 대기 시간 설정 (로그 스케일 랜덤)
MIN_SLEEP = 0.05   # 최소 대기 기본값: 50ms (빠른 처리)
MAX_SLEEP = 0.8    # 최대 대기 기본값: 800ms (패턴 회피)

print(f"📁 저장 경로: {OUTPUT_DIR}")
print(f"📅 수집 기간: {START_DATE} ~ {END_DATE}")
print(f"⏱️  대기시간: {MIN_SLEEP}~{MAX_SLEEP}초 (로그분포)")

## 1️⃣ 종목 리스트 확보

In [ ]:
print(f"🔍 {REFERENCE_DATE} 기준 KRX 전체 종목 조회 중...")

ticker_list = get_ticker_universe(REFERENCE_DATE)

print(f"✅ 총 {len(ticker_list)}개 종목 확보")

## 2️⃣ 데이터 수집 실행

⚠️ **예상 소요 시간**: 약 8~10분 (2,300개 종목 기준)

In [ ]:
# Collector 초기화
collector = RawPriceCollector(
    output_dir=OUTPUT_DIR,
    start_date=START_DATE,
    end_date=END_DATE,
    min_sleep=MIN_SLEEP,
    max_sleep=MAX_SLEEP
)

# 수집 실행
print("\n🚀 데이터 수집 시작...")
print("💡 로그 스케일 랜덤 대기로 서버 차단 회피\n")
stats = collector.collect_all(ticker_list)

print("\n" + "="*60)
print("📊 수집 완료 통계")
print("="*60)
print(f"  ✅ 성공: {stats['success']}개")
print(f"  ⚠️  데이터 없음: {stats['empty']}개")
print(f"  ❌ 실패: {stats['failed']}개")
print("="*60)

## ✅ 다음 단계

원시 데이터 수집이 완료되었습니다. 다음 단계를 진행하세요:

```bash
python scripts/02_build_features.py
```

**02단계에서 수행할 작업**:
- 기술적 지표 계산 (MA, RSI, MACD 등)
- Target 변수 생성 (수익률, 로그 등)
- 학습용 Feature Matrix 구성